<a href="https://colab.research.google.com/github/nirajlondhe8/aiml/blob/main/Intrusion_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Create a dictionary with the data
data = {
    'Duration': [0, 0, 2, 0, 1],
    'protocol_type': ['tcp', 'tcp', 'tcp', 'icmp', 'udp'],
    'service': ['http', 'private', 'smtp', 'ecr_i', 'domain_u'],
    'flag': ['SF', 'REJ', 'SF', 'SF', 'SF'],
    'src_bytes': [215, 0, 333, 0, 105],
    'dst_bytes': [45076, 0, 214, 0, 146],
    'label': ['normal', 'attack', 'normal', 'attack', 'normal']
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('network_traffic.csv', index=False)

print("Data saved to network_traffic.csv")


Data saved to network_traffic.csv


In [2]:
import pandas as pd
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load the dataset
data = pd.read_csv('network_traffic.csv')

# Preprocessing (simple encoding)
data['protocol_type'] = data['protocol_type'].astype('category').cat.codes
data['service'] = data['service'].astype('category').cat.codes
data['flag'] = data['flag'].astype('category').cat.codes

# Splitting features and target
X = data.drop('label', axis=1)
y = data['label'].apply(lambda x: 1 if x == 'normal' else 0)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Layer 1: Anomaly Detection
anomaly_detector = IsolationForest(contamination=0.1)
anomaly_detector.fit(X_train)
y_pred_anomaly = anomaly_detector.predict(X_test)

# Layer 2: Signature-based Detection
signature_classifier = RandomForestClassifier()
signature_classifier.fit(X_train, y_train)
y_pred_signature = signature_classifier.predict(X_test)

# Layer 3: Behavior-based Detection with LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X_train_reshaped = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(X_train_reshaped, y_train, epochs=5, batch_size=64, validation_split=0.2)
y_pred_behavior = model.predict(X_test_reshaped)

# Combine results for multilayered detection
# (simple majority voting or weighted combination can be used)
final_pred = (y_pred_anomaly + y_pred_signature + y_pred_behavior.flatten()) / 3
final_pred = final_pred.round()

# Evaluation
print(classification_report(y_test, final_pred))


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 0.7566 - val_accuracy: 1.0000 - val_loss: 0.6927
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0000e+00 - loss: 0.7342 - val_accuracy: 0.0000e+00 - val_loss: 0.6959
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5000 - loss: 0.7102 - val_accuracy: 0.0000e+00 - val_loss: 0.6991
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5000 - loss: 0.6844 - val_accuracy: 0.0000e+00 - val_loss: 0.7022
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5000 - loss: 0.6572 - val_accuracy: 0.0000e+00 - val_loss: 0.7054
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
